http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey=Iuowj7UW8R3d8WxSA%2Fh2wTsszZ1EZT22V%2FB%2BRBhNPW4bMCrxh0l89LSAmPjeCo1G%2FLjJTbmUbi%2FAM%2BL8Ae3cYQ%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200414

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from tqdm import tqdm
import numpy as np
from urllib.request import urlopen
import seaborn as sns
import matplotlib.pyplot as plt
import platform

# 이것만 실행하면 됨
**날짜 (총4개) 만 바꿔서 실행하면 됨.**

In [9]:
key='Iuowj7UW8R3d8WxSA%2Fh2wTsszZ1EZT22V%2FB%2BRBhNPW4bMCrxh0l89LSAmPjeCo1G%2FLjJTbmUbi%2FAM%2BL8Ae3cYQ%3D%3D'
url= 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20210115&serviceKey={}'.format(key)
#url 에서 제일 끝에 serviceKey 앞에 날짜 바꾸기1

page=requests.get(url)
soup=BeautifulSoup(page.content, "html.parser")

rows = soup.find_all('item')

rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        if i == 0:
            nameList.append(columns[j].name) 
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.to_excel("uncorrected_21-01-15_age_gender.xlsx") #날짜바꾸기2

#컬럼네임 변경
age_gen=pd.read_excel("uncorrected_21-01-15_age_gender.xlsx") #날짜바꾸기3
age_gen.rename(columns={'confcase':'확진자', 'confcaserate':'확진률', 'createdt':'등록일시분초',
                          'criticalrate':'치명률', 'death':'사망자', 'deathrate':'사망률','gubun':'구분(성별, 연령별)',
                          'seq':'확진자 성별, 연령별 고유값', 'updatedt':'수정일시분초'}, inplace=True)
age_gender=age_gen.sort_values(by= ['Unnamed: 0'], ascending=False)

age_gender.reset_index(inplace=True)
age_gender.drop('Unnamed: 0', axis=1, inplace=True)
age_gender.drop('index', axis=1, inplace=True)

#null깂 채워주기
age_gender.fillna(0, inplace=True)

#4/7 0-11 index 삭제해주기
age_gender.drop(22, inplace=True)

#최종 다운
age_gender['등록일시분초']=pd.to_datetime(age_gender['등록일시분초'], format='%Y-%m-%d')
age_gender=age_gender.reindex([ '등록일시분초','수정일시분초','구분(성별, 연령별)', '확진자','사망자','확진률','사망률', '치명률',
                               '확진자 성별, 연령별 고유값'], axis='columns')
age_gender.to_excel('corrected_21-01-15_age_gender.xlsx') #날짜바꾸기4

# 전처리 과정

## openapi 가져오기

In [2]:
key='Iuowj7UW8R3d8WxSA%2Fh2wTsszZ1EZT22V%2FB%2BRBhNPW4bMCrxh0l89LSAmPjeCo1G%2FLjJTbmUbi%2FAM%2BL8Ae3cYQ%3D%3D'
url= 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20210110&serviceKey={}'.format(key)

page=requests.get(url)
soup=BeautifulSoup(page.content, "html.parser")

rows = soup.find_all('item')

rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        if i == 0:
            nameList.append(columns[j].name) 
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.to_excel("uncorrected-_01-10_age_gender.xlsx")

## column names 바꿔주기

In [67]:
age_gen=pd.read_excel("uncorrected-_01-10_age_gender.xlsx")
age_gen.rename(columns={'confcase':'확진자', 'confcaserate':'확진률', 'createdt':'등록일시분초',
                          'criticalrate':'치명률', 'death':'사망자', 'deathrate':'사망률','gubun':'구분(성별, 연령별)',
                          'seq':'확진자 성별, 연령별 고유값', 'updatedt':'수정일시분초'}, inplace=True)
age_gender=age_gen.sort_values(by= ['Unnamed: 0'], ascending=False)
age_gender.head()

,Unnamed: 0,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
3036,3036,3905,39.90,2020-04-02 19:02:48.48,2.10,82,50.62,남성,1,NaN
3035,3035,5881,60.10,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,NaN
3034,3034,442,4.52,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,NaN
3033,3033,651,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,NaN
3032,3032,1235,12.62,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,NaN


In [68]:
age_gender.reset_index(inplace=True)
age_gender.drop('Unnamed: 0', axis=1, inplace=True)
age_gender.drop('index', axis=1, inplace=True)
age_gender

,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
0,3905,39.90,2020-04-02 19:02:48.48,2.10,82,50.62,남성,1,NaN
1,5881,60.10,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,NaN
2,442,4.52,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,NaN
3,651,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,NaN
4,1235,12.62,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,NaN
...,...,...,...,...,...,...,...,...,...
3032,9787,14.25,2021-01-10 14:21:07.99,0.09,9,0.80,40-49,6014,NaN
3033,8771,12.77,2021-01-10 14:21:07.99,0.06,5,0.44,30-39,6015,NaN
3034,10761,15.67,2021-01-10 14:21:07.99,0.00,0,0.00,20-29,6016,NaN
3035,4334,6.31,2021-01-10 14:21:07.99,0.00,0,0.00,10-19,6017,NaN


## null 값 채워주기

In [69]:
age_gender_null=age_gender.isnull()
age_gender_null.sum()

확진자                   0
확진률                   0
등록일시분초                0
치명률                   0
사망자                   0
사망률                   0
구분(성별, 연령별)           0
확진자 성별, 연령별 고유값       0
수정일시분초             3037
dtype: int64

In [70]:
age_gender.fillna(0, inplace=True)

## 4/7 의 분류기준 잘 못된 부분 수정해주기

In [71]:
#남성+여성
age_gender1=age_gender[11:13]
#0-9 ~ 80 이상 (0-11 제외)
age_gender2=age_gender[13:22]

In [72]:
age_gender1.sum()

확진자                                                       10331
확진률                                                         100
등록일시분초             2020-04-07 19:32:08.082020-04-07 19:32:08.08
치명률                                                        3.91
사망자                                                         192
사망률                                                         100
구분(성별, 연령별)                                                남성여성
확진자 성별, 연령별 고유값                                              25
수정일시분초                                                        0
dtype: object

In [73]:
age_gender2.sum()

확진자                                                            10331
확진률                                                           100.01
등록일시분초             2020-04-07 19:32:08.082020-04-07 19:32:08.0820...
치명률                                                            31.13
사망자                                                              192
사망률                                                              100
구분(성별, 연령별)              80 이상70-7960-6950-5940-4930-3920-2910-190-9
확진자 성별, 연령별 고유값                                                  162
수정일시분초                                                             0
dtype: object

확진자의 합이 같으므로 0-11 (index22) 는 삭제해줘도 무방함. 

In [74]:
age_gender.drop(22, inplace=True)

In [75]:
age_gender.head(25)

,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
0,3905,39.90,2020-04-02 19:02:48.48,2.10,82,50.62,남성,1,0.0
1,5881,60.10,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,0.0
2,442,4.52,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,0.0
3,651,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,0.0
4,1235,12.62,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,0.0
5,1851,18.91,2020-04-02 19:02:48.48,0.54,10,6.17,50-59,6,0.0
6,1312,13.41,2020-04-02 19:02:48.48,0.08,1,0.62,40-49,7,0.0
7,1012,10.34,2020-04-02 19:02:48.48,0.10,1,0.62,30-39,8,0.0
8,2656,27.14,2020-04-02 19:02:48.48,0.00,0,0.00,20-29,9,0.0
9,515,5.26,2020-04-02 19:02:48.48,0.00,0,0.00,10-19,10,0.0


In [76]:
age_gender.reset_index(inplace=True)
age_gender

,index,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
0,0,3905,39.90,2020-04-02 19:02:48.48,2.10,82,50.62,남성,1,0.0
1,1,5881,60.10,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,0.0
2,2,442,4.52,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,0.0
3,3,651,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,0.0
4,4,1235,12.62,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,0.0
...,...,...,...,...,...,...,...,...,...,...
3031,3032,9787,14.25,2021-01-10 14:21:07.99,0.09,9,0.80,40-49,6014,0.0
3032,3033,8771,12.77,2021-01-10 14:21:07.99,0.06,5,0.44,30-39,6015,0.0
3033,3034,10761,15.67,2021-01-10 14:21:07.99,0.00,0,0.00,20-29,6016,0.0
3034,3035,4334,6.31,2021-01-10 14:21:07.99,0.00,0,0.00,10-19,6017,0.0


In [77]:
age_gender.drop('index', axis=1, inplace=True)

## 잘 되었는지 확인

In [78]:
age_gender.columns

Index(['확진자', '확진률', '등록일시분초', '치명률', '사망자', '사망률', '구분(성별, 연령별)',
       '확진자 성별, 연령별 고유값', '수정일시분초'],
      dtype='object')

In [79]:
age_gender_total=pd.pivot_table(age_gender, index=['구분(성별, 연령별)'],values=['확진자', '확진률', '등록일시분초', '치명률', '사망자', '사망률',
       '확진자 성별, 연령별 고유값', '수정일시분초'], aggfunc=sum)
age_gender_total

,사망률,사망자,수정일시분초,치명률,확진률,확진자,"확진자 성별, 연령별 고유값"
"구분(성별, 연령별)",,,,,,,
0-9,0.00,0,0.0,0.00,582.13,156151,808283
10-19,0.00,0,0.0,0.00,1560.10,359914,808007
20-29,0.00,0,0.0,0.00,6298.47,1287315,807731
30-39,158.42,573,0.0,26.61,3321.91,767179,807455
40-49,285.70,1065,0.0,41.23,3714.37,848713,807179
50-59,1385.83,5148,0.0,148.61,5012.70,1139000,806903
60-69,3555.07,13613,0.0,500.84,3924.07,925728,806627
70-79,8395.31,33210,0.0,2246.89,1992.90,467001,806351
80 이상,13819.90,55360,0.0,6077.71,1193.43,274023,806075


In [80]:
age_gender_total.reset_index(inplace=True)
age_gender_total

,"구분(성별, 연령별)",사망률,사망자,수정일시분초,치명률,확진률,확진자,"확진자 성별, 연령별 고유값"
0,0-9,0.00,0,0.0,0.00,582.13,156151,808283
1,10-19,0.00,0,0.0,0.00,1560.10,359914,808007
2,20-29,0.00,0,0.0,0.00,6298.47,1287315,807731
3,30-39,158.42,573,0.0,26.61,3321.91,767179,807455
4,40-49,285.70,1065,0.0,41.23,3714.37,848713,807179
5,50-59,1385.83,5148,0.0,148.61,5012.70,1139000,806903
6,60-69,3555.07,13613,0.0,500.84,3924.07,925728,806627
7,70-79,8395.31,33210,0.0,2246.89,1992.90,467001,806351
8,80 이상,13819.90,55360,0.0,6077.71,1193.43,274023,806075
9,남성,14510.57,56991,0.0,626.42,12347.64,2867562,805523


In [81]:
age_gender_total1=age_gender_total[:9]
age_gender_total2=age_gender_total[9:]

In [86]:
age_gender.tail(11)

,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
3025,33627,48.97,2021-01-10 14:21:07.989,1.66,558,49.60,남성,6008,0.0
3026,35037,51.03,2021-01-10 14:21:07.989,1.62,567,50.40,여성,6009,0.0
3027,3432,5.00,2021-01-10 14:21:07.989,18.56,637,56.62,80 이상,6010,0.0
3028,5352,7.79,2021-01-10 14:21:07.99,5.75,308,27.38,70-79,6011,0.0
3029,10847,15.80,2021-01-10 14:21:07.99,1.21,131,11.64,60-69,6012,0.0
3030,12904,18.79,2021-01-10 14:21:07.99,0.27,35,3.11,50-59,6013,0.0
3031,9787,14.25,2021-01-10 14:21:07.99,0.09,9,0.80,40-49,6014,0.0
3032,8771,12.77,2021-01-10 14:21:07.99,0.06,5,0.44,30-39,6015,0.0
3033,10761,15.67,2021-01-10 14:21:07.99,0.00,0,0.00,20-29,6016,0.0
3034,4334,6.31,2021-01-10 14:21:07.99,0.00,0,0.00,10-19,6017,0.0


In [87]:
#남성+여성
age_gender11=age_gender[3025:3027]
#0-9 ~ 80 이상 (0-11 제외)
age_gender22=age_gender[3027:]

In [88]:
age_gender11.sum()

확진자                                                         68664
확진률                                                           100
등록일시분초             2021-01-10 14:21:07.9892021-01-10 14:21:07.989
치명률                                                          3.28
사망자                                                          1125
사망률                                                           100
구분(성별, 연령별)                                                  남성여성
확진자 성별, 연령별 고유값                                             12017
수정일시분초                                                          0
dtype: object

In [89]:
age_gender22.sum()

확진자                                                            68664
확진률                                                            99.99
등록일시분초             2021-01-10 14:21:07.9892021-01-10 14:21:07.992...
치명률                                                            25.94
사망자                                                             1125
사망률                                                            99.99
구분(성별, 연령별)              80 이상70-7960-6950-5940-4930-3920-2910-190-9
확진자 성별, 연령별 고유값                                                54126
수정일시분초                                                             0
dtype: object

## 최종 저장

In [91]:
age_gender.to_excel('corrected-01-10_age_gender.xlsx')